# **CartoCell - inference workflow (Phase 5) - BiaPy's latest version**
___  
  
**CartoCell** is a deep learning-based image processing pipeline for the high-throughput segmentation of whole epithelial cysts acquired at low resolution with minimal human intervention. The official documentation of the workflow is in [BiaPy's CartoCell tutorial](https://biapy.readthedocs.io/en/latest/tutorials/instance_seg/cartocell.html).

<figure>
<center>
<img src='https://raw.githubusercontent.com/BiaPyX/BiaPy-doc/refs/heads/master/source/img/cartocell_pipeline.png' width='800px'/>
<figcaption><b>Figure 1</b>: CartoCell processing phases (from Andrés-San Román et al., 2022).</figcaption></center>
</figure>


**This notebook replicates CartoCell's Phase 5**, i.e., it allows the segmentation of 3D epithelial cysts using a deep learning model trained on a large dataset of low-resolution cysts (see Figure 1, Phase 4, model M2).

___


**CartoCell** relies on the [BiaPy library](https://github.com/BiaPyX/BiaPy), freely available in GitHub: https://github.com/BiaPyX/BiaPy

Please note that **CartoCell** is based on a publication. If you use it successfully for your research please be so kind to cite our work:

*Andres-San Roman, J.A., Gordillo-Vazquez, C., Franco-Barranco, D., Morato, L., Fernandez-Espartero, C.H., Baonza, G., Tagua, A., Vicente-Munuera, P., Palacios, A.M., Gavilán, M.P., Martín-Belmonte, F., Annese, V., Gómez-Gálvez, P., Arganda-Carreras, I., Escudero, L.M. [CartoCell, a high-content pipeline for 3D image analysis, unveils cell morphology patterns in epithelia](https://www.cell.com/cell-reports-methods/fulltext/S2667-2375(23)00249-7). Cell Reports Methods, 3(10).*


___

<font color='red'><b>Note:</b></font> This notebook uses the **latest version of BiaPy** to replicate the pipeline proposed in the original **CartoCell** [manuscript](https://www.cell.com/cell-reports-methods/fulltext/S2667-2375(23)00249-7), thus results may vary slightly from those using the implementation of **BiaPy** at the time of publication.


___


## **Expected inputs and outputs**
___
**Inputs**

This notebook expects two folders as input:
* **Test Raw Images**: Containing the raw 3D images for testing the model. A cyst per image is expected. For optimal performance, the voxel resolution should match that of the training images: 1.62 x 1.62 x 0.5 micron/voxel.
* **[OPTIONAL] Test Label Images**: Containing the corresponding instance-label 3D images of the raw test images. Thus, their quantity and dimensions should match those of the test raw images.
* **Output Folder**: Specify a directory where the segmentation results will be saved.

**Outputs**

If the execution is successful, two folders will be created for each input image containing:
* A **TIFF image** with the cell instances before 3D Voronoi post-processing.
* A **TIFF image** with the cell instances after 3D Voronoi post-processing.


<figure>
<center>
<img src='https://biapy.readthedocs.io/en/latest/_images/cyst_sample.gif' width='300'/>
<img src='https://biapy.readthedocs.io/en/latest/_images/cyst_instance_prediction.gif' width='300'/>
<figcaption><b>Figure 2</b>: Example of input and output images. From left to rigth: 3D TIFF input image and the resulting TIFF image with the cell instances after Voronoi post-processing</figcaption></center>
</figure>


<font color='red'><b>Note</b></font>: for testing purposes, you can also run this notebook with the samples images provided in *Manage file(s) source > Option 3*. Those samples belong to the test set used in our [publication](https://www.cell.com/cell-reports-methods/fulltext/S2667-2375(23)00249-7).

**Data structure**

To ensure the proper operation of the library, the input data directory tree should be something like this:

```
dataset/
└── test
    ├── x
    │   ├── test-0001.tif
    │   ├── test-0002.tif
    │   ├── . . .
    │   └── test-9999.tif
    └── y
        ├── test-0001.tif
        ├── test-0002.tif
        ├── . . .
        └── test-9999.tif
```

**⚠️ Warning:** Ensure that images and their corresponding masks are sorted in the same way. A common approach is to fill with zeros the image number added to the filenames (as in the example).

**Input Format Support**

This notebook is compatible with a range of input formats. You can use the following file extensions: `.tif`, `.npy` (every extension for 3D images supported by [scikit-image](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread)).

## **Prepare the environment**
___

Establish connection with Google services. You **must be logged in to Google** to continue.
Since this is not Google's own code, you will probably see a message warning you of the dangers of running unfamiliar code. This is completely normal.



## **Check for GPU access**
---

By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

Go to **Runtime -> Change the Runtime type**

**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

**Accelerator: GPU** *(Graphics processing unit)*

## **Install BiaPy**
---
This might take some minutes depending on the current installed libraries in Colab.


In [ ]:
#@markdown ##Play to install BiaPy and its dependences
# Install latest release of BiaPy
!pip install biapy==3.6.1

# Then install Pytorch + CUDA 11.8
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118

# Finally install some packages that rely on the Pytorch installation
!pip install timm==1.0.14 pytorch-msssim torchmetrics[image]==1.4.*


import os
import sys
import numpy as np
from tqdm.notebook import tqdm
from skimage.io import imread
import ipywidgets as widgets
from ipywidgets import Output
from biapy import BiaPy

changed_source = False

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.

## **Manage file(s) source**
---
The input folder can be provided using three different options: by directly uploading the folder (option 1), by using a folder stored in your own Google Drive (option 2) or by automatically downloading a few samples of our data (option 3).

Depending on the option chosen, different steps will have to be taken, as explained in the following cells.


### **Option 1: Use your local files and upload them to the notebook**
---
You will be prompted to upload your files to Colab and they will be stored under `/content/input/`.

In [ ]:
#@markdown ##Play the cell to upload local files (test raw images)

from google.colab import files
import os
input_dir = '/content/input/test/x'

if os.path.exists(input_dir):
    # Ask the user if they want to delete the existing items in the folder
    delete_items = ''
    while not delete_items in ['y', 'n']:
        delete_items = input("Do you want to delete the existing items in the folder? (yes[y]/no[n]): ").strip().lower()
    if delete_items == 'y':
        for delete_root, delete_dirs, delete_files in os.walk(input_dir, topdown=False):
            for name in delete_files:
                os.unlink(os.path.join(delete_root, name))
else:
    # Ensure the directory exists
    os.makedirs(input_dir, exist_ok=True)


%cd {input_dir}
uploaded = files.upload()
%cd /content

In [ ]:
#@markdown ## [OPTIONAL] Play the cell to upload local files (test labels)

from google.colab import files
import os
input_dir = '/content/input/test/y'

if os.path.exists(input_dir):
    # Ask the user if they want to delete the existing items in the folder
    delete_items = ''
    while not delete_items in ['y', 'n']:
        delete_items = input("Do you want to delete the existing items in the folder? (yes[y]/no[n]): ").strip().lower()
    if delete_items == 'y':
        for delete_root, delete_dirs, delete_files in os.walk(input_dir, topdown=False):
            for name in delete_files:
                os.unlink(os.path.join(delete_root, name))
else:
    # Ensure the directory exists
    os.makedirs(input_dir, exist_ok=True)


%cd {input_dir}
uploaded = files.upload()
%cd /content

### **Option 2: Mount your Google Drive**
---
To use this notebook on your own data from Google Drive, you need to mount Google Drive first.

Play the cell below to mount your Google Drive and follow the link that will be shown. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

Once this is done, your data will be available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Click on the URL.

#@markdown * Sign in your Google Account.

#@markdown * Copy the authorization code.

#@markdown * Enter the authorization code.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### **Option 3: Download our publication's test samples**
---
Don't you have data readily available but still want to test the notebook? No problem! Simply execute the following cell to download a sample dataset.

Specifically, we'll use the **test set used to evaluate the M2 model** in our manuscript and publicly available in [Zenodo](https://zenodo.org/records/10973241).

In [ ]:
#@markdown ##Play to download the example dataset (publication test set)
!pip install gdown==5.1.0 --quiet
import gdown
import os

os.chdir('/content/')

print( 'Downloading original CartoCell dataset...')

gdown.download("https://drive.google.com/uc?id=1eScAB6mpmPeQcirR5xK6Rkz4AAobsG_X", "CartoCell.zip", quiet=True)

!unzip -q CartoCell.zip
!rm CartoCell.zip

print( 'Dataset downloaded and unzipped under /content/CartoCell')


Dataset downloaded and unzipped under /content/CartoCell


## **Paths for Input Images and Output Files**
___

Depending on the option you chose for managing file sources, you'll set your paths differently:

- **Option 1 (Upload from Local Machine)**:
  - Set `test_data_path` to `/content/input/test/x`
  - Set `test_data_gt_path` to `/content/input/test/y`
  
- **Option 2 (Use Google Drive Data)**:
  - Insert the paths to your input files and your desired output directory here, i.e., `/content/gdrive/MyDrive/...`.
- **Option 3 (Use Our Sample Data)**:
  - Set `train_data_path` to `/content/CartoCell/test/x`
  - Set `train_data_gt_path` to `/content/CartoCell/test/y`
  - Set `output_path` to `/content/out`

**Note**: Ensure you download your results from the `/content/out` directory after the process!

**Helpful Tip**: If you're unsure about the paths to your folders, look at the top left of this notebook for a small folder icon. Navigate through the directories until you locate your desired folder. Right-click on it and select "Copy Path" to copy the folder's path.

In [ ]:
#@markdown #####Path to test raw images
test_data_path = '/content/CartoCell/test/x' #@param {type:"string"}
#@markdown ##### Select to load ground truth labels (if exist) and measure output performance
load_gt = True #@param {type:"boolean"}
#@markdown #####If you checked the previous option, introduce here the path to the test (ground truth) labels
test_data_gt_path = '/content/CartoCell/test/y' #@param {type:"string"}
#@markdown #####Path to store the resulting images (it'll be created if not existing):
output_path = '/content/output' #@param {type:"string"}

import os
from pathlib import Path

def count_image_files(directory):
    if not directory or not os.path.exists(directory):
        return 0
    image_extensions = {'.jpg', '.jpeg', '.png', '.tif', '.npy', '.tiff', '.h5', '.hd5', '.zarr'}
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if Path(file).suffix.lower() in image_extensions:
                count += 1
    return count

num_test_images = count_image_files(test_data_path)

print(f"Number of test raw images: {num_test_images}")

if load_gt:
    num_test_labels = count_image_files(test_data_gt_path)
    print(f"Number of test label images: {num_test_labels}")
    if num_test_images != num_test_labels:
        print("Error: The number of test raw images does not match the number of test label images.")

## **Select pre-trained model to use**
---

In [ ]:
# @markdown ###OPTIONAL: Check BioImage Model Zoo (BMZ) models compatible with BiaPy
# @markdown Use this option to generate a full list of the available BiaPy-compatible models in the BMZ.

# @markdown **Important:** To select one of the listed models (if any), you will have to run the next cell and select "BioImage Model Zoo" as the source of the model. Then, paste the corresponding model's nickname into the created field.
# @markdown <div><img src="https://bioimage.io/static/img/bioimage-io-logo.svg" width="600"/></div>

import json
from pathlib import Path
import pooch
import yaml
from IPython.display import HTML, display
import logging
from biapy.models import check_bmz_model_compatibility
from packaging.version import Version
from typing import Optional, Dict, Tuple, List, Literal

# Change pooch verbosity
logger = pooch.get_logger()
logger.setLevel("WARNING")

# Extracted from BiaPy-GUI.
# Adapted from BiaPy commit: 284ec3838766392c9a333ac9d27b55816a267bb9 (3.5.2)
def check_model_restrictions(
    model_rdf,
    workflow_specs,
):
    """
    Checks model restrictions to be applied into the current configuration.

    Parameters
    ----------
    model_rdf : dict
        BMZ model RDF that contains all the information of the model.

    workflow_specs : dict
        Specifications of the workflow. If not provided all possible models will be considered.

    Returns
    -------
    option_list: dict
        Variables and values to change in current configuration. These changes
        are imposed by the selected model.
    """
    specific_workflow = workflow_specs["workflow_type"]

    # Version of the model
    model_version = Version(model_rdf["format_version"])
    opts = {}

    # 1) Change PATCH_SIZE with the one stored in the model description. This differs from the code of BiaPy where
    # get_test_inputs() is simply used as there a ModelDescr is build out of the RDF. Here we try to do it manually
    # to avoid fetching files using the network as it may be slow.
    input_image_shape = []
    if "shape" in model_rdf["inputs"][0]:
        input_image_shape = model_rdf["inputs"][0]["shape"]
        # "CebraNET Cellular Membranes in Volume SEM" ('format_version': '0.4.10')
        #   have: {'min': [1, 1, 64, 64, 64], 'step': [0, 0, 16, 16, 16]}
        if isinstance(input_image_shape, dict) and "min" in input_image_shape:
            input_image_shape = input_image_shape["min"]
    else:
        # Check axes and dimension
        input_image_shape = []
        for axis in model_rdf["inputs"][0]["axes"]:
            if 'type' in axis:
                if axis['type'] == "batch":
                    input_image_shape += [1,]
                elif axis['type'] == "channel":
                    input_image_shape += [1,]
                elif 'id' in axis and 'size' in axis:
                    if isinstance(axis['size'], int):
                        input_image_shape += [axis['size'],]
                    elif 'min' in axis['size']:
                        input_image_shape += [axis['size']['min'],]
            elif 'id' in axis:
                if axis['id'] == "channel":
                    input_image_shape += [1,]
                else:
                    if isinstance(axis['size'], int):
                        input_image_shape += [axis['size'],]
                    elif 'min' in axis['size']:
                        input_image_shape += [axis['size']['min'],]
    if len(input_image_shape) == 0:
        raise ValueError("Couldn't load input info from BMZ model's RDF: {}".format(model_rdf["inputs"][0]))
    opts["DATA.PATCH_SIZE"] = tuple(input_image_shape[2:]) + (input_image_shape[1],)

    # Capture model kwargs
    if "kwargs" in model_rdf["weights"]["pytorch_state_dict"]:
        model_kwargs = model_rdf["weights"]["pytorch_state_dict"]["kwargs"]
    elif (
        "architecture" in model_rdf["weights"]["pytorch_state_dict"]
        and "kwargs" in model_rdf["weights"]["pytorch_state_dict"]["architecture"]
    ):
        model_kwargs = model_rdf["weights"]["pytorch_state_dict"]["architecture"]["kwargs"]
    else:
        raise ValueError(f"Couldn't extract kwargs from model description.")

    # 2) Workflow specific restrictions
    # Classes in semantic segmentation
    if specific_workflow in ["SEMANTIC_SEG"]:
        # Check number of classes
        classes = -1
        if "n_classes" in model_kwargs: # BiaPy
            classes = model_kwargs["n_classes"]
        elif "out_channels" in model_kwargs:
            classes = model_kwargs["out_channels"]
        elif "classes" in model_kwargs:
            classes = model_kwargs["classes"]

        if isinstance(classes, list):
            classes = classes[0]
        if not isinstance(classes, int):
            raise ValueError(f"Classes not extracted correctly. Obtained {classes}")

        if specific_workflow == "SEMANTIC_SEG" and classes == -1:
            raise ValueError("Classes not found for semantic segmentation dir. ")
        opts["MODEL.N_CLASSES"] = max(2,classes)
    elif specific_workflow in ["INSTANCE_SEG"]:
        # Assumed it's BC. This needs a more elaborated process. Still deciding this:
        # https://github.com/bioimage-io/spec-bioimage-io/issues/621
        channels = 2
        if "output_channels" in model_kwargs:
            channels = model_kwargs["output_channels"][0]
        if channels == 1:
            channel_code = "C"
        elif channels == 2:
            channel_code = "BC"
        elif channels == 3:
            channel_code = "BCM"
        if channels > 3:
            raise ValueError(f"Not recognized number of channels for instance segmentation. Obtained {channels}")

        opts["PROBLEM.INSTANCE_SEG.DATA_CHANNELS"] = channel_code

    if "preprocessing" not in model_rdf["inputs"][0]:
        return opts

    preproc_info = model_rdf["inputs"][0]["preprocessing"]
    if len(preproc_info) == 0:
        return opts
    preproc_info = preproc_info[0]

    # 3) Change preprocessing to the one stablished by BMZ by translate BMZ keywords into BiaPy's
    # 'zero_mean_unit_variance' and 'fixed_zero_mean_unit_variance' norms of BMZ can be translated to our 'custom' norm
    # providing mean and std
    key_to_find = "id" if model_version > Version("0.5.0") else "name"
    if key_to_find in preproc_info:
        if preproc_info[key_to_find] in ["fixed_zero_mean_unit_variance", "zero_mean_unit_variance"]:
            if (
                "kwargs" in preproc_info
                and "mean" in preproc_info["kwargs"]
            ):
                mean = preproc_info["kwargs"]["mean"]
                std = preproc_info["kwargs"]["std"]
            elif "mean" in preproc_info:
                mean = preproc_info["mean"]
                std = preproc_info["std"]
            else:
                mean, std = -1., -1.

            opts["DATA.NORMALIZATION.TYPE"] = "custom"
            opts["DATA.NORMALIZATION.CUSTOM_MEAN"] = mean
            opts["DATA.NORMALIZATION.CUSTOM_STD"] = std

        # 'scale_linear' norm of BMZ is close to our 'div' norm (TODO: we need to control the "gain" arg)
        elif preproc_info[key_to_find] == "scale_linear":
            opts["DATA.NORMALIZATION.TYPE"] = "div"

        # 'scale_range' norm of BMZ is as our PERC_CLIP + 'scale_range' norm
        elif preproc_info[key_to_find] == "scale_range":
            opts["DATA.NORMALIZATION.TYPE"] = "scale_range"
            if (
                float(preproc_info["kwargs"]["min_percentile"]) != 0
                or float(preproc_info["kwargs"]["max_percentile"]) != 100
            ):
                opts["DATA.NORMALIZATION.PERC_CLIP"] = True
                opts["DATA.NORMALIZATION.PERC_LOWER"] = float(preproc_info["kwargs"]["min_percentile"])
                opts["DATA.NORMALIZATION.PERC_UPPER"] = float(preproc_info["kwargs"]["max_percentile"])

    return opts

# Check the models that BiaPy can consume
COLLECTION_URL = "https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/collection.json"
collection_path = Path(pooch.retrieve(COLLECTION_URL, known_hash=None))
with collection_path.open() as f:
    collection = json.load(f)

model_urls = [entry["rdf_source"] for entry in collection["collection"] if entry["type"] == "model"]

model_rdfs = []
for mu in model_urls:
    with open(Path(pooch.retrieve(mu, known_hash=None)), 'rt', encoding='utf8') as stream:
        try:
            model_rdfs.append(yaml.safe_load(stream))
        except yaml.YAMLError as exc:
            print(exc)

# Check axes, preprocessing functions used and postprocessing.
pytorch_models = []
imposed_vars = []

workflow_specs = {
    "workflow_type": "INSTANCE_SEG",
    "ndim": "3D",
    "nclasses": "all",
}
for model_rdf in model_rdfs:
    try:
        (
            preproc_info,
            error,
            error_message
        ) = check_bmz_model_compatibility(model_rdf, workflow_specs=workflow_specs)
    except:
        error = True

    if not error:
        if "cyst" in model_rdf["tags"]:
            model_imposed_vars = check_model_restrictions(model_rdf, workflow_specs=workflow_specs)
            imposed_vars.append(model_imposed_vars)
            pytorch_models.append(model_rdf)

# Print the possible models
html = "<table style='width:100%''>"
c = 0
for i, model in enumerate(pytorch_models):

    if 'nickname' in model['config']['bioimageio']:
        nickname = model['config']['bioimageio']['nickname']
        nickname_icon = model['config']['bioimageio']['nickname_icon']
    elif 'id' in model['config']['bioimageio']:
        nickname = model['config']['bioimageio']['id']
        nickname_icon = model['config']['bioimageio']['id_emoji']
    else:
        doi = "/".join(model['id'].split("/")[:2])
        nickname = doi
        nickname_icon = doi
    cover_url = "https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/"+nickname+"/"+str(model["version"])+"/files/"+model['covers'][0]
    restrictions = ""
    for key, val in imposed_vars[i].items():
        if key == 'MODEL.N_CLASSES':
            restrictions += "<p>number_of_classes: {}</p>".format(val)
        elif key == "PROBLEM.INSTANCE_SEG.DATA_CHANNELS":
            problem_channels = 'Binary mask + Contours'
            if val == "BC":
                problem_channels = "Binary mask + Contours"
            elif val == 'BP':
                problem_channels = "Binary mask + Central points"
            elif val == 'BD':
                problem_channels = "Binary mask + Distance map"
            elif val == 'BCM':
                problem_channels = "Binary mask + Contours + Foreground mask"
            elif val == 'BCD':
                problem_channels = "Binary mask + Contours + Distance map"
            elif val == 'BCDv2':
                problem_channels = "Binary mask + Contours + Distance map with background"
            elif val == 'Dv2':
                problem_channels = "Distance map with background"
            restrictions += "<p>problem_representation: {}</p>".format(problem_channels)
    if c == 0:
        html += "<tr>"
    html += "<td style='width:33%'>"
    html += "<p style='color:#2196f3'>%s</p><p>Nickname: %s (%s)</p>%s<img src='%s' height='200'></td>"%(
        model['name'],
        nickname,
        nickname_icon,
        restrictions,
        cover_url,
    )
    c +=1
    if c == 2:
        html += "</tr>"
        c=0
html += "</table>"
if len( pytorch_models ) == 0:
    display(HTML('<h1>No BMZ models compatible with BiaPy were found for this task.</h1><br>'))
else:
    display(HTML('<h1>List of models that can be used in BiaPy:</h1><br>'))
    display(HTML(html))


3D U-Net for cyst segmentation in fluorescenceNickname: happy-honeybee (happy-honeybee)problem_representation: Binary mask + Contours + Foreground mask,3D Squeeze-and-Excitation U-Net for cyst segmentation in fluorescenceNickname: idealistic-turtle (idealistic-turtle)problem_representation: Binary mask + Contours + Foreground mask
3D ResUNetpp for cyst segmentation in fluorescenceNickname: intelligent-lion (intelligent-lion)problem_representation: Binary mask + Contours + Foreground mask,3D Residual U-Net for cyst segmentation in fluorescenceNickname: merry-water-buffalo (merry-water-buffalo)problem_representation: Binary mask + Contours + Foreground mask
3D Residual Squeeze-and-Excitation U-Net for cyst segmentation in fluorescenceNickname: venomous-swan (venomous-swan)problem_representation: Binary mask + Contours + Foreground mask,3D Attention U-Net for cyst segmentation in fluorescenceNickname: heroic-otter (heroic-otter)problem_representation: Binary mask + Contours + Foreground mask


In [ ]:
#@markdown ###Play to select the source to build the model (BiaPy or BioImage Model Zoo) { run: "auto", vertical-output: true, display-mode: "form" }

#@markdown **BiaPy**: to use the models implemented in BiaPy.

#@markdown **Bioimage Model Zoo (BMZ)**: to use models from the [BMZ repository](https://bioimage.io/#/). You can run the above cell to generate an updated list of the models that can be used with BiaPy. Copy the nickname from the model and paste it below.
import ipywidgets as widgets
from ipywidgets import Output

changed_source = True
exists_bmz = False
# create widgets
source = widgets.ToggleButtons(
    options=['BiaPy', 'BioImage Model Zoo'],
    description='Source:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Models created during this workflow', 'BioImage Model Zoo model'],
#     icons=['check'] * 3
)

bmz = widgets.Text(
    # value='10.5281/zenodo.5764892',
    placeholder='Nickname of BMZ model',
    description='ID:',
    disabled=False
)

# display the first widget
display(source)

# intialize the output - second widget
out = Output()

def changed(change):
    '''
    Monitor change in the first widget
    '''
    global out
    global exists_bmz
    if source.value == 'BiaPy':
        bmz.layout.display = 'none'
        out.clear_output() #clear output
        out = Output() # redefine output
    else:
        bmz.layout.display = 'none'
        bmz.layout.display = 'flex'
        if not exists_bmz:
          out.append_display_data(bmz)
          display(out)
        exists_bmz = True

# monitor the source widget for changes
source.observe(changed, 'value')

ToggleButtons(description='Source:', options=('BiaPy', 'BioImage Model Zoo'), tooltips=('Models created during…

Text(value='', description='ID:', layout=Layout(display='flex'), placeholder='Nickname of BMZ model')

Output(outputs=({'output_type': 'display_data', 'data': {}, 'metadata': {}},))

## **Download pretrained model and apply 3D segmentation workflow (Phase 5)**


In [ ]:
#@markdown ##Play to download and run the pretrained model M2
import os
import errno
import yaml
import gdown

os.chdir('/content/')

job_name = "cartocell_inference_latest"
checkpoint_file = "/content/model_weights_cartocell.pth"

# remove template file it is exists
template_file = '{}.yaml'.format(job_name)
if os.path.exists( template_file ):
    os.remove( template_file )

# Download .yaml file and model weights
if not os.path.exists("cartocell_inference_latest.yaml"):
    !wget https://raw.githubusercontent.com/BiaPyX/BiaPy/refs/heads/master/templates/instance_segmentation/CartoCell_paper/cartocell_inference_latest.yaml &> /dev/null
    print("\nCartocell yaml configuration file downloaded!")

if not os.path.exists("model_weights_cartocell.pth") and source.value == "BiaPy":
    gdown.download("https://drive.google.com/uc?id=1CZbakRUOcSTRkob74PnUaNZH_rJ6CNuy", "model_weights_cartocell.pth", quiet=True)
    print( '\nM2 model weights successfully downloaded!')


# Check folders before modifying the .yaml file
if not os.path.exists(test_data_path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), test_data_path)
ids = sorted(next(os.walk(test_data_path))[2])
if len(ids) == 0:
    raise ValueError("No images found in dir {}".format(test_data_path))

if not os.path.exists(template_file):
    raise ValueError("No YAML configuration file found in {}".format(template_file))

if source.value == "BiaPy" and not os.path.exists(checkpoint_file):
    raise ValueError("No h5 checkpoint file found in {}".format(checkpoint_file))


# open template configuration file
with open( template_file, 'r') as stream:
    try:
        biapy_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# change source to build model - biapy or bmz
if changed_source:
    if source.value == "BiaPy":
        biapy_config['MODEL']['SOURCE'] = "biapy"
    elif source.value == 'BioImage Model Zoo':
        checkpoint_file = ''
        biapy_config['PATHS']['CHECKPOINT_FILE'] = ''
        biapy_config['MODEL']['LOAD_CHECKPOINT'] = False
        biapy_config['MODEL']['SOURCE'] = "bmz"
        biapy_config['MODEL']['BMZ'] = {}
        biapy_config['MODEL']['BMZ']['SOURCE_MODEL_ID'] = str(bmz.value).strip()
else:
    biapy_config['MODEL']['SOURCE'] = "biapy"


biapy_config['DATA']['TEST']['PATH'] = test_data_path
biapy_config['DATA']['TEST']['GT_PATH'] = test_data_gt_path

biapy_config['DATA']['TEST']['LOAD_GT'] = load_gt
biapy_config['TRAIN']['ENABLE'] = False
# model weights
if checkpoint_file != '':
    biapy_config['PATHS'] = {}
    biapy_config['PATHS']['CHECKPOINT_FILE'] = checkpoint_file
    biapy_config['MODEL'] = {}
    biapy_config['MODEL']['LOAD_CHECKPOINT'] = True

# save file
with open( template_file, 'w') as outfile:
    yaml.dump(biapy_config, outfile, default_flow_style=False)

print( "Inference configuration finished.")

job_name = os.path.splitext(template_file)[0].split('/')[-1]

# Run the code
biapy = BiaPy(template_file, result_dir=output_path, name=job_name, run_id=1, gpu=0)
biapy.run_job()




Cartocell yaml configuration file downloaded!
Inference configuration finished.
Date     : 2025-04-07 15:08:19
Arguments: Namespace(config='cartocell_inference_latest.yaml', result_dir='/content/output', name='cartocell_inference_latest', run_id=1, gpu=0, world_size=1, local_rank=-1, dist_on_itp=False, dist_url='env://', dist_backend='nccl')
Job      : cartocell_inference_latest_1
BiaPy    : 3.5.12
Python   : 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
PyTorch  : 2.4.0+cu118
The following changes were made in order to adapt the input configuration:
Not using distributed mode
[15:08:19.367855] Configuration details:
[15:08:19.367909] AUGMENTOR:
  AFFINE_MODE: reflect
  AUG_NUM_SAMPLES: 10
  AUG_SAMPLES: True
  BRIGHTNESS: True
  BRIGHTNESS_FACTOR: (0.1, 0.15)
  BRIGHTNESS_MODE: 3D
  CBLUR_DOWN_RANGE: (2, 8)
  CBLUR_INSIDE: True
  CBLUR_SIZE: (0.2, 0.4)
  CHANNEL_SHUFFLE: False
  CMIX_SIZE: (0.2, 0.4)
  CNOISE_NB_ITERATIONS: (1, 3)
  CNOISE_SCALE: (0.05, 0.1)
  CNOISE_SIZE: (0.2,

2025-04-07 15:08:20.850 | INFO     | bioimageio.spec._internal.io_utils:open_bioimageio_yaml:131 - loading merry-water-buffalo from https://uk1s3.embassy.ebi.ac.uk/public-datasets/bioimage.io/merry-water-buffalo/0.1.0/files/rdf.yaml
100%|█████████████████████████████████████| 3.06k/3.06k [00:00<00:00, 3.67MB/s]
100%|█████████████████████████████████████| 3.43k/3.43k [00:00<00:00, 3.92MB/s]
100%|█████████████████████████████████████| 2.05M/2.05M [00:00<00:00, 3.23GB/s]
computing SHA256 of b83b4b30e90ce0474f2164125f80696a-test-input.npy (result: dd693f8b04f5575e11f79e5433b58bc41c552ccc94ecf7b75fd892ee7c63d86c): 100%|██████████| 2048128/2048128 [00:00<00:00, 284339714.78it/s]
100%|█████████████████████████████████████| 6.14M/6.14M [00:00<00:00, 7.55GB/s]
computing SHA256 of f7d5b5c0349c414ddae78f12606fcf62-test-output.npy (result: 6e6c8ddbb6919f8db179908d6c446f6567b4d19f366a7f71dae4f66d603b5d24): 100%|██████████| 6144128/6144128 [00:00<00:00, 326314238.19it/s]
100%|███████████████████████

[15:08:27.378595] [BMZ] Overriding preprocessing steps to the ones fixed in BMZ model: {'id': 'zero_mean_unit_variance', 'kwargs': {'axes': ['channel', 'z', 'y', 'x']}}
[15:08:27.378748] [BMZ] Changed 'PROBLEM.INSTANCE_SEG.DATA_CHANNEL_WEIGHTS' from (1, 1, 1) to [1, 1, 1] as defined in the RDF
[15:08:27.378776] [BMZ] Changed 'DATA.NORMALIZATION.TYPE' from div to zero_mean_unit_variance as defined in the RDF
[15:08:27.548854] Creating normalization module . . .
[15:08:27.548950] Normalization: using mean None and std: None
[15:08:27.549031] ###########################
[15:08:27.549044] #  PREPARE INSTANCE DATA  #
[15:08:27.549054] ###########################
[15:08:27.549121] You select to create BCM channels from given instance labels and no file is detected in /content/CartoCell/test/y_BCM_thick. So let's prepare the data. This process will be done just once!
[15:08:27.549260] Creating Y_test channels . . .


100%|██████████| 1/1 [00:00<00:00,  4.54it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
100%|██████████| 1/1 [00:00<00:00,  6.27it/s]

  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                     
  0%|          | 0/1 [00:00<?, ?it/s]
                                

[15:08:52.250275] DATA.VAL.GT_PATH changed from /path/to/data to /path/to/data_BCM_thick
[15:08:52.250356] DATA.TEST.GT_PATH changed from /content/CartoCell/test/y to /content/CartoCell/test/y_BCM_thick
[15:08:52.251793] ######################
[15:08:52.251823] #   LOAD TEST DATA   #
[15:08:52.251834] ######################
[15:08:52.251861] ### LOAD ###
[15:08:52.252405] ###############
[15:08:52.252430] # Build model #
[15:08:52.252441] ###############
[15:08:53.067323] ############################
[15:08:53.067428] #  PREPARE TEST GENERATOR  #
[15:08:53.067441] ############################
[15:08:53.679574] ###############
[15:08:53.679677] #  INFERENCE  #
[15:08:53.679690] ###############
[15:08:53.679701] Making predictions on test data . . .


...


100%|██████████| 86/86 [00:00<00:00, 150.69it/s]


[15:13:17.981773] Saving (1, 133, 136, 140, 3) data as .tif in folder: /content/output/cartocell_inference_latest/results/cartocell_inference_latest_1/instance_associations



100%|██████████| 1/1 [00:00<00:00,  9.89it/s]
                                             

[15:13:18.113079] DatasetMatching: {'criterion': 'iou', 'thresh': 0.5, 'fp': 101, 'tp': 50, 'fn': 39, 'precision': 0.33112582781456956, 'recall': 0.5617977528089888, 'accuracy': 0.2631578947368421, 'f1': 0.4166666666666667, 'n_true': 89, 'n_pred': 151, 'mean_true_score': 0.3797757866677274, 'mean_matched_score': 0.6760009002685546, 'panoptic_quality': 0.28166704177856444}
[15:13:18.135361] DatasetMatching: {'criterion': 'iou', 'thresh': 0.75, 'fp': 142, 'tp': 9, 'fn': 80, 'precision': 0.059602649006622516, 'recall': 0.10112359550561797, 'accuracy': 0.03896103896103896, 'f1': 0.075, 'n_true': 89, 'n_pred': 151, 'mean_true_score': 0.07775880513566263, 'mean_matched_score': 0.7689481841193305, 'panoptic_quality': 0.057671113808949785}
[15:13:18.135436] Applying Voronoi 3D . . .



 12%|█▏        | 35207/289912 [01:24<10:23, 408.67it/s]

## **Visualize 3D instance segmentation results**
---

In [ ]:
#@markdown ###Play to visualize results from the test set
#@markdown The current model will be applied to some test images and results will be shown as browsable 2D stacks displaying:
#@markdown 1. The **Source image**.
#@markdown 2. Its corresponding **Ground truth** labels.
#@markdown 3. The model **Prediction** labels using watershed.
#@markdown 4. The model **Post-processed** labels using our Voronoi algorithm.
#@markdown 5. The **Overlay** between ground truth and post-processed labels.

%matplotlib inline
import matplotlib
import numpy as np
from numpy.random import randint, seed
from matplotlib import pyplot as plt
from ipywidgets import interact, fixed
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()

final_results = os.path.join(output_path, job_name, 'results', job_name + "_1")

instance_results = os.path.join(final_results, "per_image_instances")
post_results = os.path.join(final_results, "per_image_post_processing")

show_post = os.path.exists( post_results )

# Show a few examples to check that they have been stored correctly
ids_pred = sorted(next(os.walk(instance_results))[2])
ids_input = sorted(next(os.walk(test_data_path))[2])
ids_gt = sorted(next(os.walk(test_data_gt_path))[2])

# create random color map
vals = np.linspace(0,1,256)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.gist_rainbow(vals))
cmap.colors[0] = [0., 0., 0., 1.] # set background to black

samples_to_show = min(len(ids_input), 3)
chosen_images = np.random.choice(len(ids_input), samples_to_show, replace=False)
seed(1)

test_samples = []
test_sample_preds = []
test_sample_posts = []
test_sample_gt = []

# read 3D images again
for i in range(len(chosen_images)):
    aux = imread(os.path.join(test_data_path, ids_input[chosen_images[i]]))
    test_samples.append(aux)

    aux = imread(os.path.join(instance_results, ids_pred[chosen_images[i]])).astype(np.uint16)
    test_sample_preds.append(aux)

    if show_post:
        aux = imread(os.path.join(post_results, ids_pred[chosen_images[i]])).astype(np.uint16)
        test_sample_posts.append(aux)

    aux = imread(os.path.join(test_data_gt_path, ids_gt[chosen_images[i]])).astype(np.uint16)
    test_sample_gt.append(aux)

num_subplots = 5 if show_post else 4

# function to show results in 3D within a widget
def scroll_in_z(z, j):

    plt.figure(figsize=(25,5))
    # Source
    plt.subplot(1,num_subplots,1)
    plt.axis('off')
    plt.imshow(test_samples[j][z-1], cmap='gray')
    plt.title('Source (z = ' + str(z) + ')', fontsize=15)

    # Target (Ground-truth)
    plt.subplot(1,num_subplots,2)
    plt.axis('off')
    plt.imshow(test_sample_gt[j][z-1], cmap=cmap, interpolation='nearest')
    plt.title('Ground truth (z = ' + str(z) + ')', fontsize=15)

    # Prediction
    plt.subplot(1,num_subplots,3)
    plt.axis('off')
    plt.imshow(test_sample_preds[j][z-1], cmap=cmap, interpolation='nearest')
    plt.title('Prediction (z = ' + str(z) + ')', fontsize=15)

    # Post-processed
    if show_post:
        plt.subplot(1,num_subplots,4)
        plt.axis('off')
        plt.imshow(test_sample_posts[j][z-1], cmap=cmap, interpolation='nearest')
        plt.title('Post-processed (z = ' + str(z) + ')', fontsize=15)

    # Overlay
    plt.subplot(1,num_subplots,num_subplots)
    plt.axis('off')
    plt.imshow(test_sample_gt[j][z-1], cmap='Greens')
    plt.imshow(test_sample_posts[j][z-1], alpha=0.5, cmap='Purples')
    plt.title('Overlay (z = ' + str(z) + ')', fontsize=15)

    plt.show()

for j in range(samples_to_show):
    interact(scroll_in_z, z=widgets.IntSlider(min=1, max=test_sample_gt[j].shape[0], step=1, value=test_sample_gt[j].shape[0]//2), j=fixed(j));

In [ ]:
#@markdown ###Play to display the path to the output files (one 3D TIFF label image for each input image).

final_results = os.path.join(output_path, job_name, 'results', job_name+"_1")

instance_results = os.path.join(final_results, "per_image_instances")
post_results = os.path.join(final_results, "per_image_post_processing")

print("Output paths:")
print("    Instance segmentation files before post-processing are in {}".format(instance_results))
print("    Instance segmentation files after post-processing are in {}".format(post_results))

[09:58:54.969104] Output paths:
[09:58:54.969252]     Instance segmentation files before post-processing are in /content/output/cartocell_inference_latest/results/cartocell_inference_latest_1/per_image_instances
[09:58:54.969383]     Instance segmentation files after post-processing are in /content/output/cartocell_inference_latest/results/cartocell_inference_latest_1/per_image_post_processing


## **Download segmentation results**
---

In [ ]:
#@markdown ###Play to download a zip file with the segmentation results **before post-processing** in the validation set.

from google.colab import files

!zip -q -j /content/cartocell_val_segmentation_results_before.zip $instance_results/*.tif

files.download("/content/cartocell_val_segmentation_results_before.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@markdown ###Play to download a zip file with all **post-processed** segmentation results in the validation set.

from google.colab import files

!zip -q -j /content/cartocell_val_segmentation_results.zip $post_results/*.tif

files.download("/content/cartocell_val_segmentation_results.zip")


## **Acknowledgments**
---
We would like to express our gratitude for the inspiration drawn from the exceptional [ZeroCostDL4Mic notebooks](https://github.com/HenriquesLab/ZeroCostDL4Mic/wiki). Specifically, we've adopted some of their metric and parameter descriptions, as well as the 3D visualization widget code present in their [U-Net 3D notebook](https://colab.research.google.com/github/HenriquesLab/ZeroCostDL4Mic/blob/master/Colab_notebooks/U-Net_3D_ZeroCostDL4Mic.ipynb). Additionally, our heartfelt thanks go out to [Estibaliz Gomez-de-Mariscal](https://scholar.google.es/citations?user=gsg-TAUAAAAJ) for her invaluable support and perceptive feedback, which significantly enhanced the quality of this notebook.
